# Calculator

In [9]:
import os
import json
import numpy as np


def extract_keypoints(directory: str):
    """
        JSON 파일에서 keypoints 데이터를 추출하고 벡터화하는 함수

        1. param directory: JSON 파일이 저장된 디렉토리 경로
        2. return: 벡터화된 keypoints 데이터 리스트
    """

    # 저장소 만들기
    keypoint_list = []
    files = os.listdir(directory)

    # 디렉토리 안에 JSON 데이터만 불러오기
    for file in files:
        if file.endswith('.json'):
            file_path = os.path.join(directory, file)

            # JSON 데이터가 파일 형식이라면 data에 load하여 저장하기 
            if os.path.isfile(file_path):
                with open(file_path, "r", encoding='utf-8') as f:
                    data = json.load(f)
                    try:
                        keypoints = data['keypoints']
                        for kp in keypoints:
                            keypoint_list.append([kp['part'], kp['x'], kp['y']])
                    except KeyError:
                        print(f'데이터를 찾을 수 없습니다')

    return keypoint_list

In [12]:
directory = '../../LJH/data/front_json'
json_data = extract_keypoints(directory)
print(json_data[:17])

[['nose', 644, 148], ['left_eye', 652, 140], ['right_eye', 635, 141], ['left_ear', 664, 151], ['right_ear', 622, 151], ['left_shoulder', 679, 203], ['right_shoulder', 607, 202], ['left_elbow', 701, 227], ['right_elbow', 586, 220], ['left_wrist', 672, 220], ['right_wrist', 621, 219], ['left_hip', 671, 334], ['right_hip', 620, 335], ['left_knee', 686, 440], ['right_knee', 606, 442], ['left_ankle', 698, 545], ['right_ankle', 594, 547]]
